In [499]:
import scipy.io
import numpy as np
import CMR_IA as cmr
import matplotlib.pyplot as plt
import pickle
from statistics import mean, stdev
import pprint
# import import_ipynb
import time
import pandas as pd
import seaborn as sns

np.set_printoptions(edgeitems=20, linewidth=150,precision=4)

In [500]:
with open("../Data/simu8b_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)

In [501]:
s_mat = np.load('../Data/simu8_smat.npy')

In [502]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_rec = 0.6,
    beta_rec_new = 0,
    beta_rec_post = 0.1,
    gamma_fc = 0.4,
    gamma_cf = 0.6,
    s_fc = 0.2,
    s_cf = 2,
    c_thresh = 0.01,
    c_thresh_ass = 0.01,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.08,
    lamb = 0.04,
    eta = 0,
    omega = 8,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 16,
    d_ass = 0.5,
    No_recall = np.arange(0,8)
)
params

{'beta_enc': 0.5,
 'beta_rec': 0.6,
 'beta_cue': 0.5,
 'beta_rec_new': 0,
 'beta_rec_post': 0.1,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 2,
 's_fc': 0.2,
 'kappa': 0.08,
 'eta': 0,
 'omega': 8,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.01,
 'd_ass': 0.5,
 'lamb': 0.04,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 16,
 'max_recalls': 50,
 'learn_while_retrieving': False,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': array([0, 1, 2, 3, 4, 5, 6, 7]),
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.4,
 'gamma_cf': 0.6}

In [503]:
df_study = df_study.loc[df_study.session == 0]
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,serial_pos,list,session
0,12,24,Face12,Joe,0,0,0
1,2,29,Face2,Mark,1,0,0
2,6,27,Face6,Dan,2,0,0
3,13,18,Face13,John,3,0,0
4,11,31,Face11,George,4,0,0
5,8,19,Face8,Rob,5,0,0
6,1,30,Face1,Mike,6,0,0
7,14,22,Face14,Rich,7,0,0


In [504]:
df_test = df_test.loc[df_test.session == 0]
df_test

,test_itemno,test_item,correct_ans,study_pos,list,session
0,1,Face1,30,6,0,0
1,2,Face2,29,1,0,0
2,6,Face6,27,2,0,0
3,14,Face14,22,7,0,0
4,13,Face13,18,3,0,0
5,8,Face8,19,5,0,0
6,12,Face12,24,0,0,0
7,11,Face11,31,4,0,0


In [505]:
face_distance = np.load('../Data/simu8_distance.npy')
def get_distance(df_tmp):
    faces = np.unique(df_tmp.test_itemno)
    face_dist = {}
    for face in faces:
        this_dist = []
        for other_face in faces:
            if face != other_face:
                this_dist.append(face_distance[face-1,other_face-1])
        this_dist = np.array(this_dist)
        face_dist[face] = this_dist
    y = df_tmp.apply(lambda x: face_dist[x['test_itemno']],axis=1)
    return y
df_test['distance'] = get_distance(df_test)

distance_thresh = 3
df_test['neighbour'] = df_test.apply(lambda x: sum(x['distance'] < distance_thresh), axis=1)
df_test.drop('distance', axis=1, inplace=True)
df_test['neighbour_group'] = df_test.apply(lambda x: 6 if x['neighbour'] == 7 else x['neighbour'], axis=1)
df_test

,test_itemno,test_item,correct_ans,study_pos,list,session,neighbour,neighbour_group
0,1,Face1,30,6,0,0,5,5
1,2,Face2,29,1,0,0,5,5
2,6,Face6,27,2,0,0,5,5
3,14,Face14,22,7,0,0,6,6
4,13,Face13,18,3,0,0,6,6
5,8,Face8,19,5,0,0,5,5
6,12,Face12,24,0,0,0,1,1
7,11,Face11,31,4,0,0,3,3


In [506]:
df_thin = df_test[['list','session','test_itemno']]
df_thin = df_thin.assign(s_resp=np.nan, s_rt=np.nan)
df_thin

,list,session,test_itemno,s_resp,s_rt
0,0,0,1,NaN,NaN
1,0,0,2,NaN,NaN
2,0,0,6,NaN,NaN
3,0,0,14,NaN,NaN
4,0,0,13,NaN,NaN
5,0,0,8,NaN,NaN
6,0,0,12,NaN,NaN
7,0,0,11,NaN,NaN


In [507]:
# extarct the session data
sess = 0
list_num = len(np.unique(df_study.list))

pres_mat = df_study.loc[df_study.session == sess, ['study_itemno1', 'study_itemno2']].to_numpy()
pres_mat = np.reshape(pres_mat, (list_num, -1, 2))
pres_mat

array([[[12, 24],
        [ 2, 29],
        [ 6, 27],
        [13, 18],
        [11, 31],
        [ 8, 19],
        [ 1, 30],
        [14, 22]]])

In [508]:
cue_mat = df_thin.loc[df_thin.session == sess, 'test_itemno'].to_numpy()
cue_mat = np.reshape(cue_mat,(list_num,-1))
cue_mat

array([[ 1,  2,  6, 14, 13,  8, 12, 11]])

In [509]:
sem_mat = s_mat

In [510]:
model = cmr.CMR2(params, pres_mat, sem_mat, source_mat=None,
           rec_mat=None, ffr_mat=None, cue_mat=cue_mat, task='CR', mode='Final')

In [511]:
# model.ret_thresh

In [512]:
# model.pres_nos

In [513]:
# model.cues_indexes

In [514]:
# model.pres_indexes

In [515]:
# model.nlists

In [516]:
model.run_intrusion_cr_single_sess()

# in model:
# print(self.att_vec[self.all_nos_unique[item_idx]-1])
# print(np.dot(self.c_old[:self.nitems_unique], self.f[:self.nitems_unique].T))

In [517]:
model.nitems_unique

16

In [518]:
model.nitems_in_race

16

In [519]:
model.ret_thresh

array([inf, inf, inf, inf, inf, inf, inf, inf,  9.,  9.,  9.,  9.,  9.,  1.,  9.,  9.], dtype=float32)

In [520]:
model.rec_items

[30, 22, 27, 19, 18, 31, 24, -1]

In [521]:
model.rec_times

[1610, 2320, 2320, 2790, 2090, 2700, 3170, -1]

In [522]:
model.f_in_acc

[array([1.5356, 1.2378, 1.284 , 1.4926, 1.1571, 0.5559, 1.3823, 1.6472, 0.3927, 0.552 , 0.7046, 0.1033, 0.3015, 0.206 , 0.7365, 0.4654], dtype=float32),
 array([1.6057, 1.2782, 1.4227, 1.4862, 1.0908, 0.6329, 1.426 , 1.4996, 0.4512, 0.5053, 0.5855, 0.1937, 0.4083, 0.4879, 0.6129, 0.4664], dtype=float32),
 array([1.5833, 1.3061, 1.402 , 1.4068, 1.1853, 0.5731, 1.567 , 1.4612, 0.5144, 0.4977, 0.5177, 0.1998, 0.5886, 0.4252, 0.5431, 0.5011], dtype=float32),
 array([1.6102, 1.279 , 1.4125, 1.5749, 1.304 , 0.6044, 1.6506, 1.6448, 0.4585, 0.5321, 0.7057, 0.1497, 0.4406, 0.3082, 0.5938, 0.4919], dtype=float32),
 array([1.5612, 1.2503, 1.4584, 1.551 , 1.3715, 0.5589, 1.5921, 1.6733, 0.625 , 0.5444, 0.5987, 0.1559, 0.4392, 0.2975, 0.5548, 0.5479], dtype=float32),
 array([1.6691, 1.2457, 1.3476, 1.5639, 1.3292, 0.6419, 1.5409, 1.7589, 0.5328, 0.6874, 0.6072, 0.15  , 0.3863, 0.2687, 0.6119, 0.5445], dtype=float32),
 array([1.3632, 1.1632, 1.1358, 1.3958, 1.1875, 0.7946, 1.2739, 1.431 , 0.4759, 0.

In [523]:
# model.M_FC

In [524]:
model.M_CF

array([[0.4829, 0.8895, 0.7193, 0.7957, 0.4791, 0.296 , 0.5777, 0.4277, 0.1259, 0.2076, 0.    , 0.0698, 0.099 , 0.0808, 0.3082, 0.1614, 0.    ,
        0.    ],
       [0.7541, 0.4121, 0.6072, 0.5238, 0.1375, 0.4404, 0.4621, 0.3682, 0.    , 0.    , 0.    , 0.2109, 0.    , 0.4646, 0.    , 0.    , 0.    ,
        0.    ],
       [0.6032, 0.8209, 0.4344, 0.3033, 0.4176, 0.2611, 0.6943, 0.4964, 0.    , 0.    , 0.    , 0.1789, 0.3904, 0.2071, 0.    , 0.    , 0.    ,
        0.    ],
       [0.6096, 0.6589, 0.4547, 0.4647, 0.567 , 0.4831, 0.6707, 0.697 , 0.1633, 0.3149, 0.    , 0.0905, 0.1284, 0.1048, 0.    , 0.2093, 0.    ,
        0.    ],
       [0.3421, 0.3094, 0.6171, 0.3777, 0.4517, 0.4199, 0.7007, 0.5648, 0.2066, 0.    , 0.    , 0.1145, 0.1624, 0.1325, 0.    , 0.3272, 0.    ,
        0.    ],
       [0.1855, 0.2295, 0.0703, 0.3644, 0.2889, 0.4   , 0.1188, 0.218 , 0.    , 0.    , 0.    , 0.6   , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    ],
       [0.4498, 0.6644, 0.9005, 0.

In [525]:
recs = model.rec_items
rts = model.rec_times
result = np.column_stack((recs,rts))
df_thin.loc[df_thin.session==sess, ['s_resp','s_rt']] = result
df_simu = df_thin.merge(df_test,on=['session','test_itemno','list'])
df_simu

,list,session,test_itemno,s_resp,s_rt,test_item,correct_ans,study_pos,neighbour,neighbour_group
0,0,0,1,30.0,1610.0,Face1,30,6,5,5
1,0,0,2,22.0,2320.0,Face2,29,1,5,5
2,0,0,6,27.0,2320.0,Face6,27,2,5,5
3,0,0,14,19.0,2790.0,Face14,22,7,6,6
4,0,0,13,18.0,2090.0,Face13,18,3,6,6
5,0,0,8,31.0,2700.0,Face8,19,5,5,5
6,0,0,12,24.0,3170.0,Face12,24,0,1,1
7,0,0,11,-1.0,-1.0,Face11,31,4,3,3


In [526]:
tmp1 = df_study.study_itemno1.to_numpy()
tmp2 = df_study.study_itemno2.to_numpy()
tmp = np.concatenate((tmp1, tmp2))
tmp = np.sort(tmp)
tmp

array([ 1,  2,  6,  8, 11, 12, 13, 14, 18, 19, 22, 24, 27, 29, 30, 31])

In [527]:
testid = np.searchsorted(tmp, df_simu.test_itemno)
corrid = np.searchsorted(tmp, df_simu.correct_ans)

In [528]:
df_simu['corr_fin']= [model.f_in_acc[i][id] for i, id in enumerate(corrid)]
df_simu['othe_fin'] = [np.mean(np.delete(model.f_in_acc[i], list(range(8)) + [id])) for i, id in enumerate(corrid)]
df_simu['omax_fin'] = [np.max(np.delete(model.f_in_acc[i], list(range(8)) + [id])) for i, id in enumerate(corrid)]
df_simu

,list,session,test_itemno,s_resp,s_rt,test_item,correct_ans,study_pos,neighbour,neighbour_group,corr_fin,othe_fin,omax_fin
0,0,0,1,30.0,1610.0,Face1,30,6,5,5,0.736472,0.389362,0.704620
1,0,0,2,22.0,2320.0,Face2,29,1,5,5,0.487865,0.460463,0.612908
2,0,0,6,27.0,2320.0,Face6,27,2,5,5,0.588585,0.457007,0.543128
3,0,0,14,19.0,2790.0,Face14,22,7,6,6,0.705732,0.424957,0.593800
4,0,0,13,18.0,2090.0,Face13,18,3,6,6,0.625024,0.448333,0.598684
5,0,0,8,31.0,2700.0,Face8,19,5,5,5,0.687421,0.443052,0.611945
6,0,0,12,24.0,3170.0,Face12,24,0,1,1,0.507658,0.464943,0.561787
7,0,0,11,-1.0,-1.0,Face11,31,4,3,3,0.657214,0.458854,0.611544


In [529]:
corrid

array([14, 13, 12, 10,  8,  9, 11, 15], dtype=int64)